# 06 - NumPy: fundamentos y primeras operaciones

### Motivación inicial: cálculo de estadísticas de notas

Supongamos que disponemos de una **gran lista de alumnos** para los que tenemos las notas en varias asignaturas (por ejemplo: Matemáticas, Lengua, Historia, Física, etc.). Nuestro objetivo inicial (antes de introducir NumPy) es calcular de forma eficiente distintos **estadísticos descriptivos**:

- Media (promedio) de notas por alumno.
- Media de cada asignatura (promedio por columna).
- Mínimo y máximo (por alumno y por asignatura).
- Desviación típica (variabilidad) de la nota de cada alumno.
- Otros posibles estadísticos: mediana, rango, cuartiles, varianza, conteo de suspensos / aprobados, etc.

### Enfoque previo usando solo estructuras básicas de Python
Antes de usar NumPy, nos proponemos implementar el cálculo con diferentes representaciones para comparar legibilidad y rendimiento:

1. **Lista de listas** (`list[list[float]]`): cada fila representa a un alumno y cada posición una asignatura.
2. **Lista de tuplas** (`list[tuple[float, ...]]`): inmutable por fila, mismo significado que arriba.
3. **Lista de diccionarios** (`list[dict[str, float]]`): cada alumno es un diccionario con claves = nombres de asignaturas.

Ejemplo mínimo (3 alumnos, 3 asignaturas):

```python
# Lista de listas
notas_ll = [
    [7.5, 8.0, 6.0],   # Alumno 0
    [5.0, 9.0, 7.0],   # Alumno 1
    [8.5, 6.5, 9.0],   # Alumno 2
]

# Lista de tuplas
notas_lt = [
    (7.5, 8.0, 6.0),
    (5.0, 9.0, 7.0),
    (8.5, 6.5, 9.0),
]

# Lista de diccionarios
notas_ld = [
    {"Mat": 7.5, "Len": 8.0, "His": 6.0},
    {"Mat": 5.0, "Len": 9.0, "His": 7.0},
    {"Mat": 8.5, "Len": 6.5, "His": 9.0},
]
```

### Qué compararemos
- Código necesario para obtener cada estadístico (claridad / verbosidad).
- Facilidad para añadir una nueva asignatura o alumno.
- Robustez ante datos faltantes (por ejemplo, `None`).
- Rendimiento (tiempo) usando, por ejemplo, `timeit` sobre un dataset simulado grande.
- Uso de memoria aproximado (`sys.getsizeof` + estructuras derivadas).

### Limitaciones de las estructuras básicas
A medida que crece el volumen de datos (miles o cientos de miles de alumnos × asignaturas), los bucles anidados y conversiones manuales se vuelven:
- Más lentos (iteración pura en Python vs operaciones vectorizadas).
- Más propensos a errores (índices, alineación de columnas, claves inconsistentes).
- Menos expresivos para operaciones estadísticas más avanzadas.

Esta situación motiva la introducción de **NumPy**, que ofrece:
- Arrays homogéneos en memoria contigua (mejor localidad de caché y velocidad en C).
- Operaciones vectorizadas y broadcasting para eliminar bucles explícitos.
- Conjunto amplio de funciones estadísticas optimizadas.

En la siguiente sección, veremos por qué NumPy mejora drásticamente tanto la **claridad** como el **rendimiento** para este tipo de tareas.

In [ ]:
from __future__ import annotations
import random, math, statistics, time, json, csv, os, sys
from typing import List, Tuple, Dict

# ---------------------------------------------------------------
# Parámetros configurables
# ---------------------------------------------------------------
N_ALUMNOS = 50_000          # Ajustar para que el tiempo sea apreciable (> 30k)
ASIGNATURAS = ["Mat", "Len", "His", "Fis", "Qui", "Bio", "Ing", "Eco"]
RANGO_NOTA = (0.0, 10.0)
SEED = 42
EXPORTAR_CSV = False        # Poner True si se quiere generar un fichero CSV
N_REPETICIONES_TIMEIT = 3   # Promedia varias ejecuciones para mayor estabilidad

random.seed(SEED)

# ---------------------------------------------------------------
# 1. Generación de datos: elegimos representación Lista de Listas
#    (las otras dos variantes se dejan como ejercicio: lista de tuplas, lista de diccionarios)
# ---------------------------------------------------------------

def generar_dataset_listas(n_alumnos: int, asignaturas: List[str]) -> List[List[float]]:
    low, high = RANGO_NOTA
    return [ [random.uniform(low, high) for _ in asignaturas] for _ in range(n_alumnos) ]

notas_ll: List[List[float]] = generar_dataset_listas(N_ALUMNOS, ASIGNATURAS)

# (Ejercicios propuestos: implementar generar_dataset_tuplas y generar_dataset_diccionarios)

# ---------------------------------------------------------------
# 2. Cálculo de estadísticos sobre lista de listas
# ---------------------------------------------------------------

def estadisticos_lista_listas(notas: List[List[float]], asignaturas: List[str]):
    n_alumnos = len(notas)
    n_asigs = len(asignaturas)

    # Media por alumno
    medias_alumno = [sum(filas) / n_asigs for filas in notas]
    # Desviación típica por alumno (poblacional vs muestral -> usamos población: divisor n_asigs)
    # Para evitar repetir código, calculamos manualmente la varianza poblacional.
    desv_alumno = []
    for filas in notas:
        m = sum(filas) / n_asigs
        var = sum((x - m) ** 2 for x in filas) / n_asigs
        desv_alumno.append(math.sqrt(var))

    # Media por asignatura (iterar por columnas)
    medias_asignatura = []
    min_asignatura = []
    max_asignatura = []
    for j in range(n_asigs):
        col = [notas[i][j] for i in range(n_alumnos)]
        medias_asignatura.append(sum(col) / n_alumnos)
        min_asignatura.append(min(col))
        max_asignatura.append(max(col))

    # Otros ejemplos: mediana por asignatura
    medianas_asignatura = [statistics.median([notas[i][j] for i in range(n_alumnos)]) for j in range(n_asigs)]

    return {
        "medias_alumno": medias_alumno,
        "desv_alumno": desv_alumno,
        "medias_asignatura": medias_asignatura,
        "min_asignatura": min_asignatura,
        "max_asignatura": max_asignatura,
        "medianas_asignatura": medianas_asignatura,
    }

# ---------------------------------------------------------------
# 3. Medición de tiempos
# ---------------------------------------------------------------

def medir_tiempo(func, *args, repeticiones=N_REPETICIONES_TIMEIT, **kwargs):
    tiempos = []
    for _ in range(repeticiones):
        t0 = time.perf_counter()
        res = func(*args, **kwargs)
        t1 = time.perf_counter()
        tiempos.append(t1 - t0)
    return res, sum(tiempos) / len(tiempos), tiempos

resultados, tiempo_medio, tiempos_ind = medir_tiempo(estadisticos_lista_listas, notas_ll, ASIGNATURAS)

print(f"Alumnos: {N_ALUMNOS:,} | Asignaturas: {len(ASIGNATURAS)}")
print(f"Tiempo medio (lista de listas): {tiempo_medio:.4f} s (detalles: {[round(t,4) for t in tiempos_ind]})")

# ---------------------------------------------------------------
# 4. (Opcional) Exportar a CSV para reutilizar en otras variantes
# ---------------------------------------------------------------
if EXPORTAR_CSV:
    filename = f"notas_{N_ALUMNOS}_alumnos.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Alumno"] + ASIGNATURAS)
        for idx, fila in enumerate(notas_ll):
            writer.writerow([idx] + [f"{x:.2f}" for x in fila])
    print(f"Fichero CSV generado: {os.path.abspath(filename)}")

# ---------------------------------------------------------------
# 5. Resumen rápido de algunos resultados
# ---------------------------------------------------------------
print("Ejemplo medias (primeros 5 alumnos):", [round(x,2) for x in resultados["medias_alumno"][:5]])
print("Medias por asignatura:")
for asig, m in zip(ASIGNATURAS, resultados["medias_asignatura"]):
    print(f"  {asig}: {m:.2f}")
print("Min/Max por asignatura (pares):")
print(list(zip(ASIGNATURAS, [round(x,2) for x in resultados["min_asignatura"]], [round(x,2) for x in resultados["max_asignatura"]])))

# Nota: se deja al estudiante repetir la comparación con:
#   - Lista de tuplas
#   - Lista de diccionarios
# Y más adelante comparar con NumPy para observar la mejora significativa.


### ¿Por qué NumPy?
**Problema:** 

En este notebook aprenderás a:

- Crear **ndarrays** de NumPy y comprender sus `shape`, `dtype` y `ndim`.
- Indexar y **rebanar (slicing)** arrays.
- Usar **ufuncs** (funciones vectorizadas) y **broadcasting** básico.
- Generar datos con `numpy.random` y fijar semillas.
- Operaciones comunes: estadísticas, ordenación, concatenación.


## 1. Importación y configuración

In [3]:
import numpy as np

np.__version__

'1.26.4'

## 2. Creación de arrays

In [4]:
a = np.array([1, 2, 3])
b = np.array([[1, 2, 3], [4, 5, 6]])

z = np.zeros((2,3))
o = np.ones((3,3))
i = np.eye(3)

print("a:", a, "shape:", a.shape)
print("b:\n", b, "\nshape:", b.shape)
print("zeros:\n", z)
print("ones:\n", o)
print("eye:\n", i)

a: [1 2 3] shape: (3,)
b:
 [[1 2 3]
 [4 5 6]] 
shape: (2, 3)
zeros:
 [[0. 0. 0.]
 [0. 0. 0.]]
ones:
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
eye:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## 3. dtypes y casting

In [5]:
x = np.array([1, 2, 3], dtype=np.int32)
y = x.astype(np.float64)
x.dtype, y.dtype

(dtype('int32'), dtype('float64'))

## 4. Indexación y slicing

In [6]:
m = np.arange(12).reshape(3,4)
print("m =\n", m)
print("Fila 1:", m[1])
print("Elemento (2,3):", m[2,3])
print("Submatriz (filas 0..1, cols 1..2):\n", m[0:2, 1:3])
print("Salto de 2 en columnas:\n", m[:, ::2])

m =
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Fila 1: [4 5 6 7]
Elemento (2,3): 11
Submatriz (filas 0..1, cols 1..2):
 [[1 2]
 [5 6]]
Salto de 2 en columnas:
 [[ 0  2]
 [ 4  6]
 [ 8 10]]


## 5. Indexación booleana

In [7]:
datos = np.array([10, -3, 5, 7, -1, 0, 8])
mascara = datos > 0
print("Máscara:", mascara)
print("Positivos:", datos[mascara])

Máscara: [ True False  True  True False False  True]
Positivos: [10  5  7  8]


## 6. Ufuncs y operaciones elementwise

In [8]:
u = np.array([1,2,3,4], dtype=float)
v = np.array([0.5, 0.5, 0.5, 0.5])
print("u + v:", u + v)
print("u * v:", u * v)
print("np.sqrt(u):", np.sqrt(u))
print("np.exp(u):", np.exp(u))

u + v: [1.5 2.5 3.5 4.5]
u * v: [0.5 1.  1.5 2. ]
np.sqrt(u): [1.         1.41421356 1.73205081 2.        ]
np.exp(u): [ 2.71828183  7.3890561  20.08553692 54.59815003]


## 7. Estadísticos y agregaciones

In [9]:
M = np.arange(1,10).reshape(3,3)
print("M =\n", M)
print("Suma total:", M.sum())
print("Media por fila:", M.mean(axis=1))
print("Máximo por columna:", M.max(axis=0))

M =
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Suma total: 45
Media por fila: [2. 5. 8.]
Máximo por columna: [7 8 9]


## 8. Aleatoriedad reproducible

In [10]:
rng = np.random.default_rng(seed=42)
muestra = rng.normal(loc=0, scale=1, size=5)
muestra

array([ 0.30471708, -1.03998411,  0.7504512 ,  0.94056472, -1.95103519])

## 9. Concatenación y apilado

In [11]:
A = np.array([[1,2],[3,4]])
B = np.array([[5,6],[7,8]])
print("hstack:\n", np.hstack([A,B]))
print("vstack:\n", np.vstack([A,B]))

hstack:
 [[1 2 5 6]
 [3 4 7 8]]
vstack:
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]


## 📝 Ejercicios cortos

1) Crea un array 4x4 con los números del 0 al 15 y extrae la diagonal principal.  
2) Genera 1000 valores normales `N(0,1)` y calcula media y desviación estándar.  
3) Dado `x = [1,2,3,4,5]`, obtén un array con los cuadrados **sin usar bucles**.


hstack:
 [[1 2 5 6]
 [3 4 7 8]]
vstack:
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]
